In [1]:
using Logging, LinearAlgebra, Revise

In [2]:
using PorousMaterials, MOFun
global_logger(ConsoleLogger(stdout, Logging.Info));

┌ Info: Precompiling MOFun [top-level]
└ @ Base loading.jl:1260


# Example case: make IRMOF-1_one_ring with 2-acetylamido-terephthalate linker

## parent structure

In [3]:
parent_structure = Crystal("IRMOF-1_one_ring.cif")
strip_numbers_from_atom_labels!(parent_structure)
infer_bonds!(parent_structure, true)

┌ Info: Crystal IRMOF-1_one_ring.cif has  space group. I am converting it to P1 symmetry.
└         To afrain from this, pass `convert_to_p1=false` to the `Crystal` constructor.


## search moiety

In [4]:
s_moty = moiety("find-replace/2-!-p-phenylene");

#### nodes are sorted by order, except that R-group nodes are sent to the end of the list

Sorting the tagged nodes to the end may degrade performance for multi-nuclear replacement motifs.  This is a compromise made for the simplicity of aligning the moiety; it forces the isomorphism between the search moiety alignment mask and the replace moiety to keep a consistent node ordering with the isomorphism between the search moiety and the parent structure.

#### should write a test for tagged atoms being at the end

## replace moiety

In [5]:
r_moty = moiety("2-acetylamido-p-phenylene");

## substructure search results

In [6]:
search = s_moty ∈ parent_structure;

#### all 4 are in 1 location

## choose 1 at random for a replacement

In [7]:
s2p_isomorphism = search.results[rand(1:4)].isomorphism;

## subtract R group to get mask

In [8]:
mask = MOFun.subtract_r_group(s_moty)
infer_bonds!(mask, false)

## align mask to replace moiety

In [9]:
m2r_isomorphism = (mask ∈ r_moty).results[1].isomorphism;

#### only 1 configuration, but will not always be so (heteroleptic multi-functionalization)

## now we have the correspondence between the search moiety and our target location, and between the search moiety mask and the replace moiety.  we can perform orthogonal Procrustes

In [10]:
function adjust_for_pb!(atoms_array::Array{Atoms{Frac}})
   for atoms in atoms_array
        atoms2 = deepcopy(atoms)
        for i in 1:length(atoms.species)
            dxf = atoms.coords.xf[:, i] .- atoms.coords.xf[:, 1] # rebuild the image (fixing first node @ origin)
            nearest_image!(dxf)
            atoms2.coords.xf[:, i] = dxf
        end
        atoms= atoms2
    end
end
;

In [11]:
geometric_center(coords) = sum(coords, dims=2)[:] / 3;

In [12]:
function orthogonal_procrustes(A, B)
    F = svd(A * B')
    return F.V * F.U'
end
;

In [13]:
function perform_ops(s_moty, r_moty, xtal, s2p_isomorphism, r_moty_subset_coords, s_mask_coords)
    rot_s2p = orthogonal_procrustes(s_moty.box.f_to_c*s_moty.atoms.coords.xf,
        xtal.box.f_to_c*xtal.atoms.coords.xf[:,s2p_isomorphism])
    rot_r2m = orthogonal_procrustes(r_moty.box.f_to_c*r_moty_subset_coords,
        s_moty.box.f_to_c*s_mask_coords)
    return rot_s2p, rot_r2m
end
;

In [14]:
function shift_to_origin!(coord_arrays::Array{Array{Float64,2}})
    for array in coord_arrays
        array .-= geometric_center(array)
    end
end
;

In [15]:
function xform_r_moty(r_moty, rot_r2m, rot_s2p, xtal_subset_center, xtal)
    xformd_r_moty_coords = rot_r2m * r_moty.box.f_to_c * r_moty.atoms.coords.xf
    xformd_r_moty_atoms = Atoms{Cart}(length(r_moty.atoms.species),
        r_moty.atoms.species,
        Cart(rot_s2p * xformd_r_moty_coords .+ xtal.box.f_to_c * xtal_subset_center))
    return Frac(xformd_r_moty_atoms, xtal.box)
end
;

In [16]:
const DEBUG = true
function effect_replacement(xtal::Crystal, s_moty::Crystal, r_moty::Crystal, s2p_isomorphism::Array{Int}, m2r_isomorphism)::Crystal
    if DEBUG
        write_xyz(Cart(xtal.atoms, xtal.box), "01_xtal")
        write_xyz(Cart(r_moty.atoms, r_moty.box), "02_r_moty")
        write_xyz(Cart(s_moty.atoms, s_moty.box), "03_s_moty")
    end
    # adjust atomic coordinates to account for periodic boundaries
    xtal_subset = xtal.atoms[s2p_isomorphism] # adjust coords on this, to preserve rest of original crystal in unit cell
    xtal_subset_center = geometric_center(xtal_subset.coords.xf) # center of location for replacement
    adjust_for_pb!([xtal_subset, s_moty.atoms, r_moty.atoms])
    if DEBUG
        write_xyz(Cart(xtal_subset, xtal.box), "04_adjusted_xtal_subset")
        write_xyz(Cart(s_moty.atoms, s_moty.box), "05_adjusted_s_moty")
        write_xyz(Cart(r_moty.atoms, r_moty.box), "06_adjusted_r_moty")
    end
    
    # determine s_mask
    r_indices = MOFun.r_group_indices(s_moty) # which atoms from s_moty are NOT in r_moty?
    s_mask_indices = [index for index in 1:length(s_moty.atoms.species) if !(index ∈ r_indices)]
    s_mask_atoms = s_moty.atoms[s_mask_indices]
    s_mask_coords = s_mask_atoms.coords.xf
    if DEBUG
        write_xyz(Cart(s_moty.atoms[r_indices], s_moty.box), "07_r_group")
        write_xyz(Cart(s_mask_atoms, s_moty.box), "08_search_moiety_mask")
    end
    
    # shift coords to align centers at origin
    shift_to_origin!([s_moty.atoms.coords.xf, xtal_subset.coords.xf, s_mask_coords])
    # r_moty is different: shift all nodes according to center of a subset
    r_moty.atoms.coords.xf .-= geometric_center(r_moty.atoms.coords.xf[:, m2r_isomorphism])
    if DEBUG
        write_xyz(Cart(s_moty.atoms, s_moty.box), "09_shifted_s_moty")
        write_xyz(Cart(xtal_subset, xtal.box), "10_shifted_xtal_subset")
        write_xyz(Cart(r_moty.atoms, r_moty.box), "11_shifted_r_moty")
    end
    
    # do orthogonal Procrustes for s_moty-to-parent and mask-to-replacement alignments
    rot_s2p, rot_r2m = perform_ops(s_moty, r_moty, xtal, s2p_isomorphism,
        r_moty.atoms[m2r_isomorphism].coords.xf, s_mask_coords)
    
    # transform r_moty according to rot_r2m, rot_s2p, and xtal_subset_center, align to box
    r_coords = xform_r_moty(r_moty, rot_r2m, rot_s2p, xtal_subset_center, xtal)
    
    # subtract s_moty isomorphic subset from xtal
    keep = [i for i in 1:length(xtal.atoms.species) if !(i ∈ s2p_isomorphism)]
    crystal_species = xtal.atoms.species[keep]
    crystal_coords = xtal.atoms.coords[keep]
    
    # concatenate transformed r_moty's coords/species w/ xtal's
    species = vcat(crystal_species, r_moty.atoms.species)
    coords = hcat(crystal_coords.xf, r_coords.coords.xf)

    # return new crystal
    name = remove_extension(xtal) * "_find_" * MOFun.remove_path_prefix(s_moty.name) * "_replace_" * r_moty.name
    atoms = Atoms(species, Frac(coords))
    crystal = Crystal(name, xtal.box, atoms, Charges{Frac}(0))
    wrap!(crystal)
    infer_bonds!(crystal, true)
    return crystal
end
;

In [17]:
altered_crystal = effect_replacement(parent_structure, s_moty, r_moty, s2p_isomorphism, m2r_isomorphism)
write_cif(altered_crystal, altered_crystal.name * ".cif")